In [ ]:
import pandas as pd
import numpy as np
import torch
from transformers import AutoTokenizer,AutoModelForSequenceClassification,RobertaForSequenceClassification,RobertaModel,AutoModel

In [ ]:
train_df=pd.read_csv("/data/linqinhong/zindi/vaccine/data/Train.csv")
test_df=pd.read_csv("/data/linqinhong/zindi/vaccine/data/Test.csv")

In [ ]:
train_df.head(5)

In [ ]:
na_list=list(train_df[train_df["label"].isnull()==True].index)

In [ ]:
train_df.drop(na_list,axis=0,inplace=True)

In [ ]:
train_df[train_df["label"].isnull()==True]

In [ ]:
train_df.drop("tweet_id",inplace=True,axis=1)

In [ ]:
test_df.drop("tweet_id",inplace=True,axis=1)

In [ ]:
train_df.head(5)

In [ ]:
train_df["safe_text"]=train_df["safe_text"].apply(lambda x:x.replace("&amp",""))

In [ ]:
test_df["safe_text"]=test_df["safe_text"].apply(lambda x:str(x).replace("&amp",""))

删除带有#的，不一定能提高表现，需要对比

In [ ]:
import re
pattern1=re.compile(r"#[\w]*")

In [ ]:
train_df["safe_text"]=train_df["safe_text"].apply(lambda x:re.sub(pattern1,"",x))

In [ ]:
test_df["safe_text"]=test_df["safe_text"].apply(lambda x:re.sub(pattern1,"",x))

In [ ]:
train_df.head()

删除带有<>符号里的内容

In [ ]:
pattern2=re.compile(r"<[\w]*>")
train_df["safe_text"]=train_df["safe_text"].apply(lambda x:re.sub(pattern2,"",x))

In [ ]:
test_df["safe_text"]=test_df["safe_text"].apply(lambda x:re.sub(pattern2,"",x))

In [ ]:
train_df.head()

In [ ]:
train_df["safe_text"]=train_df["safe_text"].apply(str)

In [ ]:
test_df["safe_text"]=test_df["safe_text"].apply(str)

In [ ]:
train_df["safe_text"]=train_df["safe_text"].apply(str.lower)

In [ ]:
test_df["safe_text"]=test_df["safe_text"].apply(str.lower)

In [ ]:
train_df.head()

In [ ]:
from nltk.corpus import stopwords
stopword=stopwords.words("english")

In [ ]:
import contractions
train_df["safe_text"]=train_df["safe_text"].apply(contractions.fix)

In [ ]:
test_df["safe_text"]=test_df["safe_text"].apply(contractions.fix)

In [ ]:
def remove_stopword(text):
    return " ".join(i for i in text.split() if i not in stopword)

In [ ]:
train_df["safe_text"]=train_df["safe_text"].apply(remove_stopword)

In [ ]:
test_df["safe_text"]=test_df["safe_text"].apply(remove_stopword)

In [ ]:
train_df["label"]=train_df["label"].apply(int)

In [ ]:
train_df.reset_index(drop=True,inplace=True)

In [ ]:
modelname='bert-base-uncased'

In [ ]:
tokenizer=AutoTokenizer.from_pretrained(modelname)

In [ ]:
model=AutoModel.from_pretrained(modelname)

In [ ]:
from datasets import Dataset
train_dataset=Dataset.from_pandas(train_df)
test_dataset=Dataset.from_pandas(test_df)

In [ ]:
test_dataset=Dataset.from_pandas(test_df)

In [ ]:
def tokenize(text):
    result=tokenizer(text["safe_text"])
    return result

In [ ]:
train_dataset

In [ ]:
test_dataset

In [ ]:
train_dataset=train_dataset.map(tokenize,batched=True)

In [ ]:
test_dataset=test_dataset.map(tokenize,batched=True)

In [ ]:
train_dataset["input_ids"][0]

In [ ]:
test_dataset

In [ ]:
train_dataset=train_dataset.remove_columns("safe_text")

In [ ]:
test_dataset=test_dataset.remove_columns("safe_text")

In [ ]:
train_dataset.column_names

In [ ]:
train_dataset=train_dataset.rename_column("label","labels")

In [ ]:
def process_labels(text):
    return {"labels":text["labels"]+1}

In [ ]:
train_dataset=train_dataset.map(process_labels)

In [ ]:
train_dataset=train_dataset.remove_columns("agreement")

In [ ]:
train_dataset

In [ ]:
from sklearn.model_selection import StratifiedKFold,KFold
num_split=5
seed=3407
#SKF=StratifiedKFold(n_splits=num_split,shuffle=True)

In [ ]:
import torch
device=torch.device("cuda:1") if torch.cuda.is_available()==True else torch.device("cpu")
model.to(device)

In [ ]:
from transformers import DataCollatorWithPadding
datacollator=DataCollatorWithPadding(tokenizer=tokenizer)

In [ ]:
from torch.nn import Linear,ReLU,CrossEntropyLoss
from torch import softmax,sigmoid
"""linear1=Linear(1024,1024,bias=True).to(device)
linear2=Linear(1024,3,bias=True).to(device)
loss_fn=CrossEntropyLoss()"""

In [ ]:
"""def classifier(input,labels):
    output=linear2(linear1(input))
    output=softmax(sigmoid(output),dim=0)
    return output"""

In [ ]:
from torch import nn
import torch.nn.functional as F
class classifier(nn.Module):
    def __init__(self,input_channels,hidden,output_channels):
        super(classifier,self).__init__()
        self.linear1=nn.Linear(input_channels,hidden)
        self.linear2=nn.Linear(hidden,output_channels)
        self.batchnorm=nn.BatchNorm1d(input_channels,eps=1e-5,)
        self.dropout=nn.Dropout(0.5)
        self.relu=nn.ReLU()

    def forward(self,x):
        #x=self.dropout(x)
        hidden=self.linear1(x)
        hidden=self.batchnorm(hidden)
        active=self.relu(hidden)
        #可以添加一个残差连接https://www.kaggle.com/code/prashantkikani/nfl-starter-mlp-feature-engg
        active=self.dropout(active)
        output=self.linear2(active)
        return output


In [ ]:
Classifier=classifier(768,768,3).to(device)

In [ ]:
from torch.optim import AdamW
from transformers import get_scheduler
lr_rate=0.0005
optimizer=AdamW(Classifier.parameters(),lr=lr_rate)
#lr_scheduler=get_scheduler('linear',optimizer=optimizer,num_warmup_steps=20,num_training_steps=train_steps)

In [ ]:
from torch.utils.data import DataLoader
from torch.nn import CrossEntropyLoss
import evaluate
import math
from torch import softmax
metric_train=evaluate.load("accuracy")
metric_val=evaluate.load("accuracy")
epoches=100
total_loss=[]
from tqdm.auto import tqdm
epoch_bar=tqdm(range(epoches))
loss_fn=CrossEntropyLoss()
model.eval()
for epoch in range(epoches):
    lr_rate=0.0005/(epoch/20+1)
    optimizer=AdamW(Classifier.parameters(),lr=lr_rate)
    KF = KFold(n_splits=num_split, shuffle=True,random_state=seed+epoch)
    split_result=KF.split(train_dataset,train_dataset["labels"])
    for train_split,val_split in split_result:
        train_split_data=train_dataset.select(train_split)#.remove_columns("labels")
        #train_split_label=train_dataset.select(train_split)["labels"]
        val_split_data=train_dataset.select(val_split)#.remove_columns("labels")
        #val_split_labels=
        train_dataloader=DataLoader(train_split_data,shuffle=True,batch_size=64,collate_fn=datacollator)
        val_dataloader=DataLoader(val_split_data,shuffle=True,batch_size=16,collate_fn=datacollator)
        train_steps=len(train_dataloader)
        lr_scheduler=get_scheduler('linear',optimizer=optimizer,num_warmup_steps=0.2*train_steps,num_training_steps=train_steps)
        #train_bar=tqdm(range(len(train_dataloader)))
        #val_bar=tqdm(range(len(val_dataloader)))
        Loss=0
        for batch in train_dataloader:
            Classifier.train()
            batch={k:v.to(device) for k,v in batch.items()}
            optimizer.zero_grad()
            output=model(input_ids=batch["input_ids"],attention_mask=batch["attention_mask"])
            labels=batch["labels"]
            class_output=Classifier(output.pooler_output)
            loss=loss_fn(class_output,labels)
            Loss+=loss.item()
            loss.backward()
            optimizer.step()
            lr_scheduler.step()
            train_pred=torch.argmax(class_output,dim=1)
            metric_train.add_batch(predictions=train_pred,references=batch["labels"])
            train_acc=metric_train.compute()
            #train_bar.update(1)
        print(train_acc)
        print(Loss/len(train_dataloader))
        total_loss.append(Loss/len(train_dataloader))
            #break

In [ ]:
total_loss

In [ ]:
train_split_data

In [135]:
torch.save(Classifier.state_dict(),"/data/linqinhong/zindi/vaccine/data/MLP_bert1.pt")

In [ ]:
state_dict=torch.load("/data/linqinhong/zindi/vaccine/data/MLP_bert1.pt")
Classifier.load_state_dict(state_dict)

In [167]:
Classifier.eval()
result=[]
test_dataloader=DataLoader(test_dataset,batch_size=64,shuffle=True,collate_fn=datacollator)
for batch_test in test_dataloader:
    batch_test={k:v.to(device) for k,v in batch_test.items()}
    testoutput=model(**batch_test)
    logits=Classifier(testoutput.pooler_output)
    result.extend([(i-1)*j for i,j in zip(torch.argmax(logits,dim=1).cpu().tolist(),torch.max(softmax(logits,dim=1),dim=1).values.tolist())])

In [169]:
len(result)

5177

In [182]:
raw_test=pd.read_csv("/data/linqinhong/zindi/vaccine/data/Test.csv")

In [183]:
raw_test["target"]=result

In [189]:
raw_test

,tweet_id,target
0,00BHHHP1,0.997660
1,00UNMD0E,0.999457
2,01AXPTJF,0.678370
3,01HOEQJW,-0.829689
4,01JUKMAO,0.000000
...,...,...
5172,ZXVVNC5O,0.000000
5173,ZYIANVI8,0.000000
5174,ZYITEHAH,0.000000
5175,ZZ3BMBTG,0.690118


In [188]:
raw_test.drop("safe_text",axis=1,inplace=True)

In [190]:
raw_test.to_csv("/data/linqinhong/zindi/vaccine/data/result.csv",index=None)